# Safe and efficient radius fo the obstacles in the Quasiperiodic Array

At this point, we are able to simulate the dynamic of a particle inside of any Quasiperiodic Array, but this is only true if each of the Voronoi's Polygons associated to each vertex of the Quasiperiodic Array contain one and only one of the obstacles.

The problem now is find out which is the maximum radius possible that ensures us that hypothesis. In this notebook, we will make an algorithm that give us that radius.

The general idea is the next one:

1.- Generate a Square Patch centered in a random point in the plane.

2.- Get the Voronoi's Polygons structure to the vertices given by the Square Patch algorithm.

3.- Calculate the minimum distance between the vertex of the Quasiperiodic Array and the sides of his container polygon.

4.- Iterate that calculation over all the vertices in the Square Patch and retain only the minimum value.

5.- Iterates over a given number of Square Patches.

In [1]:
#The following packages are required:
using Plots
using DelimitedFiles
using LinearAlgebra

#The following files are required:
include("Operaciones_Basicas.jl")
include("Red_Dual.jl")
include("Separacion_Arreglos_X_Y.jl")
include("Generador_Vecindades_Pto_Arbitrario.jl")
include("voronoi.jl")
include("Algoritmo_Voronoi.jl")
include("Algoritmo_Cluster_Principal_Iterado.jl")
include("Algoritmo_Parche_Cuadrado.jl")
include("Algoritmo_Radio_Seguro_Eficiente_Obstaculos.jl")

radio_Eficiente_Seguro_Iterado (generic function with 1 method)

##### System Information

In [2]:
#Information of the Quasiperiodic Array
NSides = 5; #Symmetry of the quasiperiodic lattice
Error_Margin = 2; #Margin of error in the GDM of the possible integers
Star_Vectors = [[BigFloat(1),0]]; #Array wich will contain the Star Vectors
for i in 1:(NSides-1)
    push!(Star_Vectors, [cos((2*i)*pi/NSides), sin((2*i)*pi/NSides)]); #Complete the Star_Vector Array
end
Alphas_Array = fill(0.2, NSides); #Array of the alphas constant
Average_Distance_Stripes = fill(NSides/2, NSides); #Array with the average distance between stripes

#Let's read the information of the almost-circular neighborhoods of the system
Patch_Information = readdlm("Informacion_Parche_N_$(NSides).csv", BigFloat);
Reduction_Factor = 0.8; #The factor with which we multiple the Average Radius to generate the Safe Radius
Average_Radius = Patch_Information[end]; #The value of the previously calculated "Average Radius"
Safe_Radius = Reduction_Factor*Average_Radius; #The Safe Radius value

## One single Square Patch

##### Square Patch Generation

In [3]:
#Generate a square patch of the Quasiperiodic Array around the Initial Position
X,Y,APoint_Initial,Centroids,Dictionary_Centroids = parche_Cuadrado(Patch_Information,Patch_Information[2],Reduction_Factor,Average_Distance_Stripes,Star_Vectors,Alphas_Array);

In [4]:
#=
plot()
for i in 1:4:length(X)
    plot!([X[i],X[i+1],X[i+2],X[i+3],X[i]],[Y[i],Y[i+1],Y[i+2],Y[i+3],Y[i]], markersize = 0.2, key = false, aspect_ratio=:equal, grid = false, color =:black)
end
plot!()
scatter!([APoint_Initial[1]], [APoint_Initial[2]], color = "red")
scatter!([APoint_Initial[1]+sqrt(2)*Safe_Radius], [APoint_Initial[2]+sqrt(2)*Safe_Radius], color = "red")
scatter!([APoint_Initial[1]-sqrt(2)*Safe_Radius], [APoint_Initial[2]+sqrt(2)*Safe_Radius], color = "red")
scatter!([APoint_Initial[1]-sqrt(2)*Safe_Radius], [APoint_Initial[2]-sqrt(2)*Safe_Radius], color = "red")
scatter!([APoint_Initial[1]+sqrt(2)*Safe_Radius], [APoint_Initial[2]-sqrt(2)*Safe_Radius], color = "red")
=#

##### Voronoi's Polygons structure for the Vertices of the Square Patch

In [5]:
#Obtain the vertices of all the polygons as duples.
Sites_Vertices = [(Float64(X[i]), Float64(Y[i])) for i in 1:length(X)];
unique!(Sites_Vertices); #Eliminate all the copies of a vertex

#Get the Voronoi structure with only the non-repeated vertices
Voronoi_Vertices = getVoronoiDiagram(Sites_Vertices);

#Let's generate a Dictionary with the coordinates of the vertices that lay inside the square cluster.
#The Dictionary relates "Vertex (X,Y) -> "true" or "false" depending if the vertex is inside the square or not
Dictionary_Vertices_Inside_Square = Dict();
for i in Sites_Vertices
    if (APoint_Initial[1]-sqrt(2)*Safe_Radius <= i[1] <= APoint_Initial[1]+sqrt(2)*Safe_Radius) && (APoint_Initial[2]-sqrt(2)*Safe_Radius <= i[2] <= APoint_Initial[2]+sqrt(2)*Safe_Radius)
        Dictionary_Vertices_Inside_Square[i] = true;
    else
        Dictionary_Vertices_Inside_Square[i] = false;
    end
end

##### Visualization of the Square Patch vertices

In [6]:
X_Unique,Y_Unique = separacion_Arreglo_de_Arreglos_2D(Sites_Vertices);

In [7]:
#=
plot()
for i in 1:length(X_Unique)
    scatter!([X_Unique[i]], [Y_Unique[i]], color = "black", markersize = .25, aspectratio=:equal, leg = false)
end
plot!()
=#

##### Obtain the safe radius of a single square patch

In [8]:
Safe_Obstacle_Radius = radio_Eficiente_Seguro(Voronoi_Vertices, Dictionary_Vertices_Inside_Square)

0.30307676954024876

## Iterates over many differents square patches

In [9]:
Number_Square_Patch = 10; #The number of different Square Patch that desire to look for.

Safe_Obstacle_Radius = radio_Eficiente_Seguro_Iterado(Number_Square_Patch,Patch_Information,Reduction_Factor,Average_Distance_Stripes,Star_Vectors,Alphas_Array)

The algorithm is calculating the Minimum Distance in the Square Patch 1
The algorithm is calculating the Minimum Distance in the Square Patch 2
The algorithm is calculating the Minimum Distance in the Square Patch 3
The algorithm is calculating the Minimum Distance in the Square Patch 4
The algorithm is calculating the Minimum Distance in the Square Patch 5
The algorithm is calculating the Minimum Distance in the Square Patch 6
The algorithm is calculating the Minimum Distance in the Square Patch 7
The algorithm is calculating the Minimum Distance in the Square Patch 8
The algorithm is calculating the Minimum Distance in the Square Patch 9
The algorithm is calculating the Minimum Distance in the Square Patch 10


0.2938926248195979